In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### We have a dataset that contains tweets from people from different countries at the time of the Covid-19 crisis, classified into several categories, such as negative, positive, and neutral. We will analyze and then train a model to make predictions.

###  السلام عليكم لدينا مجموعة بيانات تحتوي علي تغريدات لاشخاص من دول مختلفة في وقت ازمة كوفيد19 مصنفعة لعدة تصنيفات مثل سلبي وايجابي ومحايد سنوم بتحليل ثم تدريب نموذج علي  التنبأ 

In [ ]:
# استردراد المكتبات الضرورية 
#import  necessary libraries
# استيراد مكتبة باندا
import pandas as pd
# Importing the pandas library
# استيراد مكتبة نمباي
import numpy as np
# Importing the numpy library
# استيراد وظيفة word_tokenize من مكتبة NLTK
from nltk import word_tokenize
# Importing the word_tokenize function from the NLTK library
# استيراد مكتبة stopwords من مكتبة NLTK
from nltk.corpus import stopwords
# Importing the stopwords library from the NLTK library
# استيراد مكتبة spacy
import spacy 
# Importing the spacy library
# تحميل موديل اللغة الإنجليزية من مكتبة spacy
nlp = spacy.load("en_core_web_sm") 
# Loading the English language model from the spacy library
# تجاهل الإنذارات
import warnings
warnings.filterwarnings('ignore')
# Ignoring warnings
# استيراد مكتبة re
import re
# Importing the re library
# استيراد مكتبة string
import string
# Importing the string library
import nltk
# تنزيل قائمة الكلمات الوقفية من مكتبة NLTK

# Downloading the list of stopwords from the NLTK library
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
from nltk.corpus import stopwords          
from nltk.tokenize import TweetTokenizer

nltk.download('stopwords')
nltk.download('wordnet')

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#في العربية: هذه المكتبة توفر وظيفة لتقسيم البيانات إلى مجموعات التدريب والاختبار
# This library provides a function for splitting the data into training and testing sets.
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Flatten,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

In [ ]:
# قراءة مجموعة البيانات الاختبارية من المسار المحدد
# Reading the test dataset from the specified path
df = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv')

# قراءة مجموعة البيانات التدريبية من المسار المحدد مع ترميز 'ISO-8859-1'
# Reading the train dataset from the specified path with 'ISO-8859-1' encoding
df1 = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv', encoding='ISO-8859-1')

# عرض إطار البيانات
# Displaying the DataFrame
df

In [ ]:
df1

### We analyze and explore data In this process, we will work on df1 because the two files have the same data and columns, with only different numbers


In [ ]:
# استخدام الدالة info() لعرض معلومات عن إطار البيانات df1
# Using the info() function to display information about the DataFrame df1
df1.info()

In [ ]:
# استخدام الدالة describe() لعرض إحصائيات وصفية عن إطار البيانات df1
# Using the describe() function to display descriptive statistics about the DataFrame df1
df1.describe()

In [ ]:
df1.Sentiment.value_counts()

In [ ]:
df1.TweetAt.value_counts()

### We will now process the empty values ​​and then visualize the data

In [ ]:
df['Location'] = df['Location'].str.replace('Israel ??', '')
df1['Location'] = df1['Location'].str.replace('Israel ??', '')

In [ ]:
# استخدام الدالة isnull() لتحديد القيم الناقصة في إطار البيانات df1
df1.isnull().sum()

In [ ]:
# استخدام الدالة isnull() لتحديد القيم الناقصة في إطار البيانات df
df.isnull().sum()

In [ ]:
# استخدام الدالة dropna() لحذف الصفوف التي تحتوي على قيم مفقودة في إطار البيانات df1
df.dropna(inplace=True)
df1.dropna(inplace=True)

In [ ]:
values = df1['Sentiment'].unique()
counts = df1['Sentiment'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=values, y=counts)
plt.xlabel('Values')
plt.ylabel('Counts')
plt.title('Bar Chart of Sentiment')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))  # تحديد عرض وارتفاع التصوير بالبوصة
plt.pie(counts, labels=values, autopct='%1.1f%%')
plt.title('Pie Chart of Sentiment')
plt.show()

### We find here that the Neutral ratio is the most among all and the least is Extremely Negative 

In [ ]:
dates = df1['TweetAt'].unique()
values1 = df1['TweetAt'].value_counts()

In [ ]:
dates = pd.to_datetime(dates)

# إنشاء مخطط خطي
plt.figure(figsize=(12, 6))
plt.plot(dates, values1, marker='o')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Line Plot of Values over Time')
plt.grid(True)
plt.show()


### This chart shows us that tweets related to Covid-19 were high in the beginning, but as we got used to it, the number of tweets decreased.


In [ ]:
# إنشاء مخطط عمودي
plt.figure(figsize=(40, 30))
plt.bar(dates, values1, color='green')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Bar Plot of Values over Time')
plt.grid(True)
plt.show()

### Here the number for each day is explained in detail



In [ ]:
#In searching the Places column, we find that we have 1717 different places
#في البحث في عامود الاماكن نجد ان لدينا 1717 مكان مختلف 
df.Location.value_counts()

In [ ]:

# إنشاء مخطط الانتشار
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Sentiment', y='Location', data=df[20:60])
plt.xlabel('Column 1')
plt.ylabel('Column 2')
plt.title('Scatter Plot of Column 1 vs Column 2')
plt.grid(True)
plt.show()


### With the creation of the chart, we find that there are invalid and almost imaginary values ​​that must be eliminated
### مع انشاء المخطط نجد ان هناك قيم غير صالحه وشبه وهمية يجب التخلص منها 


In [ ]:
print(df.columns)

In [ ]:
# First, we will convert the entire column to text because there are some values ​​in the middle that are not text
# اولا سنقوم بتحويل العامود بالكامل الي نص لان هناك بعض القيم في الوسط ليست نصوص
df['Location'] = df['Location'].astype(str)

In [ ]:
import re

# قراءة البيانات إلى DataFrame
# على سبيل المثال، لنفترض أن لديك DataFrame يسمى df والعمود الذي تريد تنظيفه هو "Location"

# دالة لإزالة الرموز غير المرغوب فيها باستخدام مكتبة re
def remove_special_characters(text):
    # تعريف التعبير العادي لإزالة الرموز
    pattern = r'[^a-zA-Z\s]'   # يتم الاحتفاظ بالأحرف الأبجدية الإنجليزية، الأرقام، والفراغات
 
    # استخدام الدالة sub() لإزالة الرموز باستخدام التعبير العادي
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

# تطبيق الدالة على عمود "Location"
df['Location'] = df['Location'].apply(remove_special_characters)


In [ ]:

# إنشاء مخطط الانتشار
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Sentiment', y='Location', data=df[20:60])
plt.xlabel('Column 1')
plt.ylabel('Column 2')
plt.title('Scatter Plot of Column 1 vs Column 2')
plt.grid(True)
plt.show()

### The data was set and the visualization was successful


### Now, to make it easier to visualize the Sentiment column better, we will convert it to numbers
### الان ليسهل تصور عامود Sentiment بشكل افضل سنقوم بتحويله الي ارقام

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(df1['Sentiment'], bins=10)
plt.xlabel('Sentiment')
plt.ylabel('Frequency')
plt.title('Histogram of Sentiment')
plt.show()

In [ ]:
# قائمة بالتطابق بين القيم النصية والقيم العددية المقابلة
mapping = {
    'Extremely Negative': -2,
    'Negative': -1,
    'Neutral': 0,
    'Positive': 1,
    'Extremely Positive': 2
}

# استخدام دالة map() لتحويل القيم النصية إلى القيم العددية
df1['Sentiment_1'] = df1['Sentiment'].map(mapping)

In [ ]:
df1

In [ ]:
df1['Location'] = df1['Location'].apply(remove_special_characters)

In [ ]:
# إنشاء مخطط الصندوق والشارب
plt.figure(figsize=(6, 15))
sns.boxplot(x='Sentiment_1', y='Location', data=df1[20:80])
plt.xlabel('Category')
plt.ylabel('Value')
plt.title('Boxplot of Category vs Value')
plt.grid(True)
plt.show()

In [ ]:
print(len(df1['Sentiment']), len(df1['Location']))

In [ ]:
# التحقق من القيم المفقودة في العمود
missing_values = df1['Location'].isnull()

# استخدام العملية ~ لتحديد الصفوف التي لا تحتوي على قيم مفقودة
valid_values = ~missing_values

# حذف الصفوف التي تحتوي على القيم الجغرافية
df1 = df1[valid_values] 


In [ ]:
# تحويل القيم في عمود البيانات إلى نصوص
df1['Location'] = df1['Location'].astype(str)

# استخدام تعبير الاسترجاع النصي لتحديد الصفوف التي تحتوي على القيم الجغرافية
geographic_values = df1['Location'].str.contains(r'\b\d+\.\d+,-\d+\.\d+\b', regex=True)

# حذف الصفوف التي تحتوي على القيم الجغرافية
df1 = df1[~geographic_values]


In [ ]:
df1

In [ ]:
df1['Sentiment'].value_counts()

In [ ]:
import re

# قراءة البيانات إلى DataFrame
# على سبيل المثال، لنفترض أن لديك DataFrame يسمى df والعمود الذي تريد تنظيفه هو "Location"

# دالة لإزالة الرموز غير المرغوب فيها باستخدام مكتبة re
def remove_special_characters(text):
    # تعريف التعبير العادي لإزالة الرموز
    pattern = r'[^a-zA-Z\s]'   # يتم الاحتفاظ بالأحرف الأبجدية الإنجليزية، الأرقام، والفراغات
     
    # استخدام الدالة sub() لإزالة الرموز باستخدام التعبير العادي
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

# تطبيق الدالة على عمود "Location"
df['Location'] = df['Location'].apply(remove_special_characters)

In [ ]:

# تحميل قائمة الكلمات الوقفية
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# قائمة لتخزين النصوص المنظفة
cleaned_texts = []

# تنظيف النصوص
for text in df1['OriginalTweet']:
    # تحويل النص إلى حروف صغيرة
    text = text.lower()
    
    # إزالة الرموز الغير مفيدة
    text = text.translate(str.maketrans('', '', string.punctuation + '@#$'))
    # إزالة الروابط
    text = re.sub(r'http\S+', '', text)
    # تقسيم النص إلى كلمات
    words = word_tokenize(text)
    
    # إزالة الكلمات الوقفية
    words = [word for word in words if word not in stop_words]
    
    # إعادة تجميع الكلمات لتكوين النص النظيف
    cleaned_text = ' '.join(words)
    
    # إضافة النص النظيف إلى القائمة
    cleaned_texts.append(cleaned_text)

# إنشاء DataFrame جديد يحتوي على النصوص المنظفة

df1['OriginalTweet_cleaned'] = cleaned_texts

# طباعة النتائج أو حفظها في ملف CSV
print(df1.head())

# لحفظ البيانات المنظفة في ملف CSV
# cleaned_df.to_csv('مسار_الملف_المنظف.csv', index=False)

In [ ]:
df1

In [ ]:
#32567
#32567
#32567
df1 = df1.reset_index()
df1

In [ ]:
for i in range(5,25):
    print(f"{i+1}: {df1['OriginalTweet_cleaned'][i]} -> {df1['Sentiment'][i]}")

In [ ]:
def choosingcol(df1):
    new_df=df1.loc[:, ['OriginalTweet_cleaned', 'Sentiment']]
    return new_df

#------------------------------------------------------------------
def reducing_sentiment(df1):
    df1 = df1.loc[df1['Sentiment'] != 'Neutral']

    # replace extremely positive and extremely negative with positive and negative, respectively
    df1['Sentiment'] = df1['Sentiment'].replace({'Extremely Positive': 'Positive', 'Extremely Negative': 'Negative'})
    
    return df1
#-----------------------------------------------------------------
def converting_tonumeric(df):
    sentiment_dict = {'Positive': 1, 'Negative': 0}
    df1['Sentiment'] = df1['Sentiment'].replace(sentiment_dict)
    return df1

In [ ]:
# Using functions to make train_set ready 
df1 = choosingcol(df1)
df1 = reducing_sentiment(df1)
df1 = converting_tonumeric(df1)
df1 = df1.reset_index(drop=True)
df1 

In [ ]:
contractions_dict = {
    "ain't": "am not / are not / is not / has not / have not",
    "aren't": "are not",
    "can't": "cannot",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'll": "he shall / he will",
    "he's": "he is",
    "I'd": "I had / I would",
    "I'll": "I shall / I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'll": "it shall / it will",
    "it's": "it is",
    "let's": "let us",
    "might've": "might have",
    "mightn't": "might not",
    "must've": "must have",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she had / she would",
    "she'll": "she shall / she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they had / they would",
    "they'll": "they shall / they will",
    "they're": "they are",
    "they've": "they have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'll": "we shall / we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "where's": "where is",
    "who'd": "who had / who would",
    "who'll": "who shall / who will",
    "who're": "who are",
    "who's": "who is",
    "who've": "who have",
    "won't": "will not",
    "would've": "would have",
    "wouldn't": "would not",
    "you'd": "you had / you would",
    "you'll": "you shall / you will",
    "you're": "you are",
    "you've": "you have"
}

In [ ]:
def cleaning_with_re(tweet):
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet = re.sub(r'https?://(?:www\.[^\s\n\r]+|[^\s\n\r]+)', '', tweet)

    # remove hashtag (#)
    tweet = re.sub(r'#', '', tweet)
    
    #replaces newline (\n) and carriage return (\r) characters in a tweet with an empty string
    tweet = re.sub(r'[\n\r]', '', tweet)
    
    #replaces the numbers with an empty string
    tweet = re.sub(r'\d+', '', tweet)
    
    return tweet

#----------------------------------------------------------------
def expanding_words(tweet):
    
    words = tweet.split()
    expanded_words = []
    for word in words:
        if word.lower() in contractions_dict:
            expanded_words.extend(contractions_dict[word.lower()].split("/"))
        else:
            expanded_words.append(word)
    return " ".join(expanded_words)

#----------------------------------------------------------------
def tokenizing(tweet):
    
    stopwords_english = stopwords.words('english') 
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True)
    new_tweet = tokenizer.tokenize(tweet)
    
    clean_tweet=[]
    
    for word in new_tweet: # Going through every word in tokens list
        if (word not in stopwords_english and word not in string.punctuation):  # remove punctuation and stopwords
            clean_tweet.append(word)

    return clean_tweet

#----------------------------------------------------------------
def stemming(tweet):
    # Instantiate stemming class
    stemmer = PorterStemmer() 

    # Create an empty list to store the stems
    stemmed_tweets = [] 

    for word in tweet:
        stem_word = stemmer.stem(word)  # stemming word
        stemmed_tweets.append(stem_word)  # append to the list
        
    return stemmed_tweets

In [ ]:
def preprocessing(tweet):
    
    tweet=cleaning_with_re(tweet)
    tweet=expanding_words(tweet)
    tweet=tokenizing(tweet)
    tweet=stemming(tweet)
    
    return tweet

In [ ]:
def build_freqs(listoftweets, labels):
    
    labelslist = np.squeeze(labels).tolist()
    
    # Start with an empty dictionary and populate it by looping over all tweets and all the words in them
    freqs = {}
    for y, tweet in zip(labels, listoftweets):
        t=preprocessing(tweet)
        for word in t:
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
                
    # Calculate the sum of frequencies for each word
    word_sums = {}
    for (word, y), count in freqs.items():
        if word in word_sums:
            word_sums[word] += count
        else:
            word_sums[word] = count

    # Divide each key of word and class by the sum
    for (word, y), count in freqs.items():
        freqs[(word, y)] /= word_sums[word]
            
    return freqs

#------------------------------------------------------------------
def extract_features(tweet, freqs, preprocessing=preprocessing):
    
    # process_tweet tokenizes, stems, and removes stopwords
    word_list = preprocessing(tweet)
    
    # 2 elements for [ positive, negative] counts
    features = np.zeros(2) 

    # loop through each word in the list of words
    for word in word_list:
        
        # increment the word count for the positive label 1
        features[0] += freqs.get((word, 1.0),0)
            
        # increment the word count for the negative label 0
        features[1] += freqs.get((word, 0),0)
    
    features = features[None, :] 
    assert(features.shape == (1, 2))
    return features

In [ ]:
%%time 
word_frequencies=build_freqs(df1['OriginalTweet_cleaned'].tolist(), df1['Sentiment'].tolist())

In [ ]:
X = np.zeros((len(df1['Sentiment']), 2))
for i in range(len(df1['Sentiment'])):
    X[i, :]= extract_features(df1['OriginalTweet_cleaned'][i], word_frequencies)

print(X)
    
# training labels corresponding to X
Y = np.array(df1['Sentiment'])

In [ ]:
# Model 
model = Sequential()
model.add(Dense(4, input_dim=2, activation='relu'))  # Hidden layer with 4 neurons
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Output layer with logistic regression

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=11, batch_size=32)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df1.OriginalTweet_cleaned)

# عدد الكلمات الفريدة
num_unique_words = len(tokenizer.word_index)
print("Number of unique words:", num_unique_words)

In [ ]:
# إضافة عمود جديد يحتوي على طول كل جملة
df1['sentence_length'] = df1['OriginalTweet_cleaned'].apply(lambda x: len(x.split()))

# طباعة أول عشر صفوف للتحقق
df1.head(10)

In [ ]:
df1.describe()

In [ ]:
vocab_size = 54228  
max_length = 100 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1['Sentiment'] = le.fit_transform(df1['Sentiment'])

In [ ]:
# تقسيم البيانات إلى مجموعات الاختبار والتدريب
#X_train, X_test, y_train, y_test = train_test_split(df1['OriginalTweet_cleaned'], df1['Sentiment'], test_size=0.2, random_state=42)

# مثال على تحويل النص إلى متغيرات قابلة للتحليل
#vectorizer = CountVectorizer()
#X_train_vectorized = vectorizer.fit_transform(X_train)
#X_test_vectorized = vectorizer.transform(X_test)

# بناء وتدريب النموذج
#model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
#model.fit(X_train_vectorized, y_train)

# التنبؤ باستخدام النموذج
#predictions = model.predict(X_test_vectorized)

# قياس أداء النموذج
#accuracy = accuracy_score(y_test, predictions)
#print("Accuracy:", accuracy)

In [ ]:
output_dim = len(df1['Sentiment'].unique())  # Assuming 'label' is the column containing class labels
output_dim

In [ ]:
#from sklearn.preprocessing import LabelEncoder
#lb = LabelEncoder()
#df1.Sentiment = lb.fit_transform(df1.Sentiment)

In [ ]:
#x = df1.drop(['index','UserName','Location','TweetAt','OriginalTweet','Sentiment_1','Sentiment','ScreenName'],axis=1)
x = df1['OriginalTweet_cleaned']
y = df1['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x.shape
vocab_size = 10000  
max_length = 100